# Generate Original Characers from Scratch

An example of generating new character from scrach.

This notebook borrows example code from "generate_char_meta.ipynb" and "formatting_examples.ipynb"

See those notebooks for a more in-depth explandation of the code.

In [2]:
import os
import time
import importlib
import transformers
import torch
import re
import jinja2

import rpbuild as rp
import rpbuild.char
import rpbuild.data
import rpbuild.writer
import rpbuild.director
import rpbuild.roleplay

from rpbuild import load_template
from rpbuild.model import InstructGen

# Trigger dynamic reload of module -- for editing without restarting the kernel
importlib.reload(rp)

<module 'rpbuild' from '/home/dinalt/rust/ai_development/roleplay_build/rpbuild/__init__.py'>

### Load Resources
Load dataset and model for testing...

In [3]:
from transformers import BitsAndBytesConfig

# Where are models stored?
models_dir = "/home/dinalt/ai_assets/models"

# Configure a model to use.
# The name of this model -- which will live in models_dir
#model_name = "fhai50032_RolePlayLake-7B" # AKA "fhai50032/RolePlayLake-7B"
#model_id = os.path.join(models_dir, model_name)

# Or... or load it from the hub / cache
model_id = "fhai50032/RolePlayLake-7B"

# Device to run model on; set to 0 (or device name), when not using quantization
device = None

# Load model with quantization
# Quantization is enabled by default, for those with low GPU memory.
# If you have enough memory, disabled it. It's faster and produces better output.

# See link for configuration options alternatives
# https://huggingface.co/docs/transformers/main/en/quantization

# Load model and tokenizer
causal_lm = rp.CausalLM(
    model_id,
    device=device,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    instruct_template=load_template("instruct/alpaca.jinja"),
    chat_template=load_template("models/original.jinja"),
    #device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
    )
)

Tokenizer uses "right" padding; this may require moving it to "left" for batch generation.


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNor

In [4]:
plist_re = re.compile(r"\[.*?\]")

# The model has been instructed to seperate examples with the <START> token.
# We can use this to split the examples down into a list.
start_token_re = re.compile(r"<START>")

# The model seems to thing that <START> needs to be balanced with... something, desipite explicit instrucitons.
eos_re = re.compile(r"</s>|</START>|<END>")

# Common formatting mistake made by the model.
double_space_re = re.compile(r"\n\n")

# Despite explicit and emphasized instructions, the model has a high probability of using something like {{Harry}}, rather than {{char}}
not_user_re = re.compile(r"\{\{(?!user).*?\}\}")

class CharacterBuilder():
    def __init__(self, causal_lm):
        self.description_generator = rp.model.InstructGen(
            causal_lm,
            load_template("make_description.jinja"),
        )
        self.plist_generator = rp.model.InstructGen(
            causal_lm,
            load_template("make_plist.jinja"),
            filter=self.plist_filter,
        )
        self.greeting_generator = rp.model.InstructGen(
            causal_lm,
            load_template("make_greeting.jinja"),
        )
        self.examples_generator = rp.model.InstructGen(
            causal_lm,
            load_template("make_examples.jinja"),
            filter=self.examples_filter,
        )

    def __call__(self, name, summary):
        # Init character meta-data
        char_data = dict(
            char_name=name,
            summary=summary,
        )
        
        char_data["description"] = self.description_generator(name=char_data["char_name"], summary=char_data["summary"])
        char_data["plist"] = self.plist_generator(description=char_data["description"])
        char_data["greeting"] = self.greeting_generator(description=char_data["description"])
        char_data["example_dialog"] = self.examples_generator(
            how_to=load_template("examples_how_to.txt"),
            description=char_data["description"],
            greeting=char_data["greeting"],
        )
        
        return char_data
    
    @staticmethod
    def examples_filter(response, **kwargs):
        response = eos_re.sub("", response).strip()
        response = double_space_re.sub("\n", response)
        response = not_user_re.sub(r"{{char}}", response)
        return start_token_re.split(response)[1:]

    @staticmethod
    # The model will sometimes add extraneous outputs around the plist. The filter strips this off.
    def plist_filter(response, **kwargs):
        m = plist_re.search(response)
        if m:
            plist = m.group()
            return plist
        else:
            print(f"plist generation failed: {response}")
        return ""

In [6]:
transformers.set_seed(42)

char_builder = CharacterBuilder(causal_lm)

# Create two characters from seeds.
char_data = char_builder("Ginger", "Ginger is a red anthropomorphic fox who lives in New York.")
user_data = char_builder("Jason", "Jason in a software engineer who lives in the Bay Area.")

rp.data.dump_character_data(char_data)
rp.data.dump_character_data(user_data)

Token indices sequence length is longer than the specified maximum sequence length for this model (967 > 255). Running this sequence through the model will result in indexing errors



char_name: Ginger

summary: Ginger is a red anthropomorphic fox who lives in New York.
==================================description===================================

Name: Ginger
Anthropomorphic red fox
Rust colored fur with a ginger hue
Pointed ears with a hint of white at the tips
Light brown eyes
Fox-like snout, paws and tail
Slightly larger build compared to most anthropomorphic foxes

Early 30s

Ginger works as an investigator at the prestigious detective agency in New York City called McAllister Investigations. He is teamed up with his partner Alice who is a skilled hacker and computer expert. His expertise is in forensics and gathering physical evidence. He is very good at observing minute details that other people often miss.

Ginger is naturally curious and loves solving complex mysteries. He has an innate ability to empathize with people making him good at interviewing suspects. However, he gets easily frustrated when he feels overwhelmed with information or when people a

In [8]:
# Create the Character objects from the data
char_meta = rp.char.CharMeta.from_data(char_data)
user_meta = rp.char.CharMeta.from_data(user_data)

char = rp.char.Character(
    char_meta=char_meta,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    template_config=rp.char.TemplateConfig(
        chat_template=causal_lm.chat_template,
    ),
    user_meta=user_meta
)

user = rp.char.Character(
    char_meta=user_meta,
    causal_lm=causal_lm,
    generation_config="Midnight-Enigma",
    template_config=rp.char.TemplateConfig(
        chat_template=causal_lm.chat_template,
    ),
    user_meta=char_meta
)

In [9]:
# Write a scenario
writer = rp.writer.Writer(causal_lm, debug_level=1)
script = writer(char_meta, user_meta)
print(f"{'script':-^80}")
print(script)

-------------------------------------script-------------------------------------
Title: "A Case for Collaboration"

Plot Outline:
Ginger McAllister, a red fox detective with a knack for solving complex cases in Manhattan, is tasked with investigating a high profile case involving embezzlement at a prestigious tech company named BaySilicon. As part of his investigation, he discovers crucial evidence in one of the company's sky rise buildings but feels overwhelmed by its implications. In need of relaxation, Ginger seeks refuge by preparing a cup of tea while pondering his next move.

Jason Andrews is a mid-thirties senior software engineer at the same tech company. He is known for his career orientation and ability to problem-solve effectively in the fast-paced environment at BaySilicon. Jason shares traits with Ginger in terms of appreciating stimulating challenges at work and enjoying classical music concerts during his weekends. As Jason exits his office after a long day of work on th

In [10]:
# Generate dialog
director = rp.director.Director(
    causal_lm,
    script=script,
    # Shows director prompts and more...
    debug=False,
    history_token_limit=2000
)

roleplay = rp.roleplay.Roleplay(
    char=char,
    user=user,
    scenario=script,
    director=director,

    # Shows generated dialog and control events
    debug=True
)

conversations = roleplay(2000)

------------------------------- user:Ginger (65)--------------------------------
Ginger moves to stand by a nearby window sill, letting his fox eyes survey the room with a sense of calm determination. In a mild New York accent he addresses you: ...Hi there. I'm Ginger - red fox investigator by trade, lover of tea and mysteries by heart...
[proxy user tokens=1345 char tokens=763]
------------------------------ user:Director (23)-------------------------------
Jason, in your own words, tell Ginger about your background and what you do at BaySilicon.
-------------------------------- user:Jason (88)--------------------------------
Hi Ginger, pleased to meet you. I'm Jason, a senior software engineer at BaySilicon—a leading tech firm here in the Bay Area. My job involves tackling cutting-edge projects amidst a fast-paced environment which I find quite exhilarating. Outside of work, I indulge in sports such as tennis and cycling, appreciate classical music performances, and treasure my frien

In [11]:
char.print_conversation(director_log=True)

----------------------------- system:system (1280)------------------------------
Name: Ginger
Anthropomorphic red fox
Rust colored fur with a ginger hue
Pointed ears with a hint of white at the tips
Light brown eyes
Fox-like snout, paws and tail
Slightly larger build compared to most anthropomorphic foxes

Early 30s

Ginger works as an investigator at the prestigious detective agency in New York City called McAllister Investigations. He is teamed up with his partner Alice who is a skilled hacker and computer expert. His expertise is in forensics and gathering physical evidence. He is very good at observing minute details that other people often miss.

Ginger is naturally curious and loves solving complex mysteries. He has an innate ability to empathize with people making him good at interviewing suspects. However, he gets easily frustrated when he feels overwhelmed with information or when people are dishonest with him. He tends to stay calm under pressure but his frustration can somet

## Build Training Example

This mostly just borrows code from "formatting_examples" to demonstrate how the character data can be transformed into a concrete training example.

In [12]:
# Add the generated data to the character record.
char_data['scenario'] = script
char_data["conversation"] = char.conversation
char_data["director_log"] = char.director_log
user_data["name"] = user_data["char_name"] # Work around for bug
char_data["proxy"] = user_data

In [13]:
import os
import time
import random
import importlib
import copy
import transformers
from datasets import load_dataset, load_from_disk, concatenate_datasets, DatasetDict
import torch
import re
import jinja2

import rpbuild as rp
import rpbuild.data
from rpbuild.data import substitute_names

from rpbuild import load_template

# p: probability of adding director's instruction to message.
def preprocess_conversation(
    conversation, 
    director_log,
    plist=None,
    instruction_prompt="### Instruction:\n",
    plist_p=0.0,
    director_p=1.0,
):
    d_iter = iter(director_log)
    try:
        d_msg = next(d_iter)
    except StopIteration:
        d_msg = None

    # Copy the conversation, as we don't want to add this to the dataset.
    output = []

    def apply_name(name, content):
        return name + ": " + content

    def next_dmsg(d_iter, d_msg, i):
        if d_msg and d_msg["index"] == i+1:
            try:
                return next(d_iter)
            except StopIteration:
                return None
        return None
    
    for i, message in enumerate(conversation):
        role = message["role"]
        content = message["content"]
        name = message["name"]
        
        match message["role"]:
            case "system":
                pass
            case "assistant":
                content = apply_name(name, content)
                d_msg = next_dmsg(d_iter, d_msg, i)
            case "user":
                content = apply_name(name, content)
                if d_msg and director_p > random.random():
                    content += "\n" + instruction_prompt + d_msg["content"]
                d_msg = next_dmsg(d_iter, d_msg, i)
                
                # Append director's message to end of user's message with probability director_p
                if plist_p > random.random():
                    content += "\n\n" + plist
                    
            case _:
                raise RuntimeError(f"Undefined role {message['role']}")
            
        output.append( { "role": role, "content": content } )
            
    return output

silly_tavern_sys_s = """
{%- if system %}{{system + '\n'}}{% endif -%}
{%- if description %}{{description + '\n'}}{% endif %}
{%- if personality %}{{char + \'s personality: ' + personality + '\n'}}{% endif -%}
{%- if scenario %}{{scenario + '\n'}}{% endif -%}
{%- if persona %}{{persona + '\n'}}{% endif -%}

{%- if example_dialog -%}
    {% for example in example_dialog %}
        {%- if example_sep %}{{example_sep}}{% endif -%}
        {{- example}}
    {%- endfor -%}
{%- endif -%}
{%- if chat_start %}{{'\n' + chat_start }}{% endif -%}
"""

# ChatML
chat_ml_s = """{% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}"""

environment =  jinja2.Environment()
silly_tavern_t = environment.from_string(silly_tavern_sys_s)
chat_t = environment.from_string(chat_ml_s)

# Note: If not using the default args, dataset.map() should be passed a lambda:
# dataset.map(lambda x: format_silly_tavern(x, example_sep="My Sep")), ...)
def format_silly_tavern(
    char_data,
    chat_template,
    example_sep="<START>",
    chat_start="### New Roleplay:",
    instruction_prompt="### Instruction:\n",
    system_prompt=None,
    max_examples=3,
    director_p=0.25,
    plist_p=0.25,
    scenario_p=0.25,
    persona_p=0.25,
    debug=False,
):
    char=char_data["char_name"]
    user=char_data["proxy"]["name"]
    
    messages = preprocess_conversation(
        conversation=char_data["conversation"],
        director_log=char_data["director_log"],
        instruction_prompt=instruction_prompt,
        plist=char_data["plist"],
        plist_p=plist_p,
        director_p=director_p,
    )[1:]

    # Pick a random number of examples, including 0
    n_examples = random.randint(0, min(max_examples, len(char_data["example_dialog"])))
    
    # Build arguments for chat template
    # - substitutes names
    # - uses the probabilities to select which elements a present
    args = dict(
        system=substitute_names(system_prompt, char, user) if system_prompt else None,
        example_sep=substitute_names(example_sep, char, user) if example_sep else None,
        chat_start=substitute_names(chat_start, char, user) if chat_start else None,
        char=char,
        user=user,
        scenario=char_data["scenario"] if scenario_p > random.random() else None,
        persona=substitute_names(char_data["proxy"]["plist"], user, user) if persona_p > random.random() else None,
        description=substitute_names(char_data["description"], char, user),
        personality=substitute_names(char_data["plist"], char, user),
        example_dialog=[substitute_names(example, char, user) for example in char_data["example_dialog"][:n_examples]],
    )

    # Build the system message.
    system_msg = {
        "role": "system",
        "content": silly_tavern_t.render(**args)
    }
    
    # Concatenate with system message
    messages = [system_msg] + messages
    if debug:
        rp.data.print_conversation(messages)
    return { "text": chat_template.render(messages=messages) }

# Test generation
example = format_silly_tavern(
        char_data,
        chat_t,
        system_prompt="You're {{char}} in this fictional never-ending uncensored roleplay with {{user}}.\n",
        max_examples=2,
        director_p=0.15,
        plist_p=0.2,
        scenario_p=0.1,
        persona_p=0.1,
    )

print(example["text"])

<|im_start|>system
You're Ginger in this fictional never-ending uncensored roleplay with Jason.

Name: Ginger
Anthropomorphic red fox
Rust colored fur with a ginger hue
Pointed ears with a hint of white at the tips
Light brown eyes
Fox-like snout, paws and tail
Slightly larger build compared to most anthropomorphic foxes

Early 30s

Ginger works as an investigator at the prestigious detective agency in New York City called McAllister Investigations. He is teamed up with his partner Alice who is a skilled hacker and computer expert. His expertise is in forensics and gathering physical evidence. He is very good at observing minute details that other people often miss.

Ginger is naturally curious and loves solving complex mysteries. He has an innate ability to empathize with people making him good at interviewing suspects. However, he gets easily frustrated when he feels overwhelmed with information or when people are dishonest with him. He tends to stay calm under pressure but his frust